# Datasets Validation using Grandinet Boosting Classifier:

# For the ArticularyWordRecognition Dataset:

In [1]:
from sktime.datasets import load_UCR_UEA_dataset
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
from sktime.datatypes._panel._convert import from_nested_to_3d_numpy
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from pyts.datasets import load_basic_motions
from sklearn.model_selection import train_test_split

/Users/dinsind/.local/lib/python3.8/site-packages/sktime/datatypes/_series/_check.py:43: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_INDEX_TYPES = (pd.Int64Index, pd.RangeIndex, pd.PeriodIndex, pd.DatetimeIndex)
/Users/dinsind/.local/lib/python3.8/site-packages/sktime/datatypes/_hierarchical/_check.py:50: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_INDEX_TYPES = (pd.Int64Index, pd.RangeIndex, pd.PeriodIndex, pd.DatetimeIndex)
/Users/dinsind/.local/lib/python3.8/site-packages/sktime/datatypes/_hierarchical/_check.py:51: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_MULTIINDEX_TYPES = (pd.Int64Index, pd.RangeIndex)
/Users/dinsind/.local/lib/pyth

In [2]:
X_train,y_train= load_UCR_UEA_dataset('ArticularyWordRecognition', split="train", return_X_y=True)

In [3]:
X_test,y_test= load_UCR_UEA_dataset('ArticularyWordRecognition', split="test", return_X_y=True)

In [4]:
X_3d = from_nested_to_3d_numpy(X_train)
X_3d.shape

(275, 9, 144)

In [5]:
X_test_3d= from_nested_to_3d_numpy(X_test)
X_test_3d.shape

(300, 9, 144)

In [6]:
class reservoir_features:
    '''
    creates an object associated with a multivariate dataset
    
    
    '''
    def __init__(self,data,num_features):
        '''
        data: Takes in a multidimensional array (x * y * z) - z>y
        Initializes it
        x: Timeseries
        y: Attributes for a given timeseries observation
        z: timestamped observations (features)
        
        num_features: you must specify the dimension you want to reduce it to
        
        '''
        self.features = []
        self.filters_used = []
        self.original_data = data.copy()
        self.data = data.copy()
        self.num_features = num_features
        self.x = data.shape[0]
        self.y = data.shape[1]
        self.z = data.shape[2]
        # perform checks 
        #1. 3d numpy array
        #2. Each time series should have same number of observations
        #3. num_features should be less than timestamped observations
    
    def normalize(self):
        '''
        Each attribute could potentially be on a different scale
        modifies the original data and performs a min max normalization
        '''
        for i in range(self.original_data.shape[0]):
            for j in range(self.original_data.shape[1]):
                self.data[i][j] = (self.original_data[i][j] - self.original_data[i][j].min())/(self.original_data[i][j].max()-self.original_data[i][j].min())
    
    
    def filters(self,stride_len = [1], num_filters = 1):
        '''
        stride_len: num of columns to skip after each filter multiplication
        num_filters: you can specify the number of filters you need; each filter will be of a differnt size
        size of filter = n*m 
        (n = # of rows = attribute size, 
        m = # of columns)
        '''
        #Have error check to make sure stride len is a list and value is <length of attributes
        n = self.y
        
        
        #Edge case vals is empty/smaller than num_filters
        
        for iteration in range(num_filters):
            m = self._get_m(stride_len[iteration])
            filter_a = np.random.random((n,m))
            print("filter of size ", str(n), "*", str(m), "was created\n")
            self.filters_used.append(filter_a)
            
            temp_features =[]
            for i in range(self.x):
                temp = []
                j = 0
                while j + m < self.data.shape[2]:
                    temp.append((filter_a*self.data[i,:,j:j+m]).mean())
                    j+=stride_len[iteration]
                temp_features.append(temp)
            self.features.append(temp_features)
    
    
    def _get_m(self,stride_len):
        '''
        stride_len: 
        based on stride length,& num_features, we calculate possible filter size 
        '''
        m = self.z -(self.num_features)*stride_len
        return m
    
    
    def result_features(self):
        '''
        if multiple filters were added, takes the average result
        '''
        ans =[]
        for timeseries in range(len(self.features[0])):
            temp =[]
            for feature in range(len(self.features[0][0])):
                val = np.mean([self.features[filter][timeseries][feature] for filter in range(len(self.features))])
                temp.append(val)
            ans.append(temp)
        return ans
        
        

In [7]:
def transform_data(data, num_features, stride_len, num_filters):
    data_transformed = reservoir_features(data ,num_features = num_features)
    #normalize
    data_transformed.normalize()
    #create 2 filters
    data_transformed.filters(stride_len = stride_len, num_filters = num_filters)
    data_transformed = data_transformed.result_features()
    return data_transformed

In [8]:
X_train_transformed= transform_data(X_3d, 40, [3], 1)


filter of size  9 * 24 was created



In [9]:
X_test_transformed=transform_data(X_test_3d, 40, [3], 1)

filter of size  9 * 24 was created



In [10]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                  max_depth=1, random_state=0).fit(X_train_transformed, y_train)
clf.score(X_train_transformed, y_train)

0.07272727272727272

In [11]:
clf.apply(X_train_transformed)

array([[[1., 2., 1., ..., 1., 1., 1.],
        [2., 2., 2., ..., 2., 2., 1.],
        [2., 2., 2., ..., 2., 1., 2.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[1., 2., 1., ..., 1., 1., 1.],
        [2., 2., 2., ..., 2., 2., 1.],
        [2., 2., 2., ..., 2., 1., 2.],
        ...,
        [1., 2., 1., ..., 1., 1., 1.],
        [1., 2., 1., ..., 1., 1., 1.],
        [1., 2., 1., ..., 1., 1., 1.]],

       [[1., 2., 1., ..., 1., 1., 1.],
        [1., 2., 2., ..., 2., 1., 1.],
        [1., 2., 2., ..., 2., 1., 2.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       ...,

       [[2., 2., 1., ..., 2., 2., 1.],
        [2., 2., 2., ..., 2., 2., 1.],
        [2., 2., 2., ..., 2., 2., 2.],
        ...,
        [1., 2., 1., ..., 2., 2., 1.],
        [1., 2., 1., ..., 2., 2., 1.],
        [1., 2., 1., ..., 2., 2.

In [12]:
clf.predict_log_proba(X_train_transformed)

/Users/dinsind/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:1268: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)


array([[  0., -inf, -inf, ..., -inf, -inf, -inf],
       [-inf,   0., -inf, ..., -inf, -inf, -inf],
       [  0., -inf, -inf, ..., -inf, -inf, -inf],
       ...,
       [-inf,   0., -inf, ..., -inf, -inf, -inf],
       [-inf,   0., -inf, ..., -inf, -inf, -inf],
       [-inf,   0., -inf, ..., -inf, -inf, -inf]])

## Accuracy for Train Data:

In [13]:
# Accuracy:
clf_train_pred=clf.predict(X_train_transformed)
clf_acc_score = accuracy_score(y_train, clf_train_pred)
print(clf_acc_score)


0.07272727272727272


In [14]:
roc_auc_score(y_train, clf.predict_proba(X_train_transformed), multi_class='ovr')

0.5170454545454546

In [15]:
train_f1 = f1_score(y_train, clf_train_pred, average='weighted')
train_f2 = f1_score(y_train, clf_train_pred, average='macro')
train_f3 = f1_score(y_train, clf_train_pred, average='micro')
print(train_f1)
print(train_f2)
print(train_f3)

0.04489134274801395
0.04489134274801394
0.07272727272727272


## For the Test set:

In [16]:

clf_test_pred=clf.predict(X_test_transformed)
clf_test_acc_score = accuracy_score(y_test, clf_test_pred)
print(clf_test_acc_score)

0.05


In [17]:
roc_auc_score(y_test, clf.predict_proba(X_test_transformed), multi_class='ovr')

0.5052083333333334

In [18]:
#f1-score:
test_f1 = f1_score(y_test, clf_test_pred, average='weighted')
test_f2 = f1_score(y_test, clf_test_pred, average='macro')
test_f3 = f1_score(y_test, clf_test_pred, average='micro')
print(test_f1)
print(test_f2)
print(test_f3)

0.018380287708303657
0.01838028770830366
0.05000000000000001


# For the AtrialFibrillation dataset:

In [19]:
X_AF,y_AF= load_UCR_UEA_dataset('AtrialFibrillation', split="train", return_X_y=True)
X_AF_test,y_AF_test= load_UCR_UEA_dataset('AtrialFibrillation', split="test", return_X_y=True)
X_AF_3d = from_nested_to_3d_numpy(X_AF)
X_AF_3d.shape
X_AF_test_3d= from_nested_to_3d_numpy(X_AF_test)
X_AF_test_3d.shape

print(X_AF_3d.shape)
print(X_AF_test_3d.shape)

(15, 2, 640)
(15, 2, 640)


In [20]:
X_train_transformed_AF = transform_data(X_AF_3d, 40, [3], 1)
X_test_transformed_AF = transform_data(X_AF_test_3d, 40, [3], 1)

filter of size  2 * 520 was created

filter of size  2 * 520 was created



In [21]:
print(len(X_train_transformed_AF))
print(len(X_train_transformed_AF[0]))

15
40


In [22]:
clf_AF = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                  max_depth=1, random_state=0).fit(X_train_transformed_AF, y_AF)
clf_AF.score(X_train_transformed_AF, y_AF)

1.0

In [23]:
clf_AF.apply(X_train_transformed_AF)

array([[[2., 2., 1.],
        [2., 2., 2.],
        [1., 1., 1.],
        ...,
        [1., 2., 1.],
        [2., 2., 2.],
        [2., 1., 2.]],

       [[1., 1., 1.],
        [2., 1., 2.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [2., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [2., 1., 2.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [2., 1., 1.],
        [1., 1., 1.]],

       ...,

       [[2., 2., 1.],
        [2., 2., 2.],
        [2., 1., 2.],
        ...,
        [2., 2., 2.],
        [2., 2., 2.],
        [2., 1., 2.]],

       [[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.],
        ...,
        [2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]],

       [[2., 1., 1.],
        [2., 1., 2.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [2., 1., 1.],
        [2., 1., 2.]]])

## Accuracy for Train Data:


In [24]:
# Accuracy:
clf_train_pred_AF=clf_AF.predict(X_train_transformed_AF)
clf_acc_score_AF = accuracy_score(y_AF, clf_train_pred_AF)
print("Accuracy_score:",clf_acc_score_AF)

train_AF_f1 = f1_score(y_AF, clf_train_pred_AF, average='weighted')
train_AF_f2 = f1_score(y_AF, clf_train_pred_AF, average='macro')
train_AF_f3 = f1_score(y_AF, clf_train_pred_AF, average='micro')
print("f1_score:Weighted:",train_f1)
print("f1_score:macro:",train_f2)
print("f1_score:micro:",train_f3)


Accuracy_score: 1.0
f1_score:Weighted: 0.04489134274801395
f1_score:macro: 0.04489134274801394
f1_score:micro: 0.07272727272727272


In [25]:
roc_auc_score(y_AF, clf_AF.predict_proba(X_train_transformed_AF), multi_class='ovr')

1.0

## Accuracy for Test Data:

In [26]:
# Accuracy:
clf_test_pred_AF=clf_AF.predict(X_test_transformed_AF)
clf_acc_score_test_AF = accuracy_score(y_AF_test, clf_test_pred_AF)
print("Accuracy_score:",clf_acc_score_test_AF)

# F1-Score:
test_AF_f1 = f1_score(y_AF_test, clf_test_pred_AF, average='weighted')
test_AF_f2 = f1_score(y_AF_test, clf_test_pred_AF, average='macro')
test_AF_f3 = f1_score(y_AF_test, clf_test_pred_AF, average='micro')
print("f1_score:Weighted:",test_AF_f1)
print("f1_score:macro:",test_AF_f2)
print("f1_score:micro:",test_AF_f3)

Accuracy_score: 0.26666666666666666
f1_score:Weighted: 0.2261904761904762
f1_score:macro: 0.2261904761904762
f1_score:micro: 0.26666666666666666


In [27]:
roc_auc_score(y_AF_test, clf_AF.predict_proba(X_test_transformed_AF), multi_class='ovr')

0.42

# For the Cricket dataset

In [28]:
X_C,y_C= load_UCR_UEA_dataset('Cricket', split="train", return_X_y=True)
X_C_test,y_C_test= load_UCR_UEA_dataset('Cricket', split="test", return_X_y=True)
X_C_3d = from_nested_to_3d_numpy(X_C)
X_C_3d.shape
X_C_test_3d= from_nested_to_3d_numpy(X_C_test)
X_C_test_3d.shape

print(X_C_3d.shape)
print(X_C_test_3d.shape)

(108, 6, 1197)
(72, 6, 1197)


In [29]:
X_train_transformed_C = transform_data(X_C_3d, 200, [3], 1)
X_test_transformed_C = transform_data(X_C_test_3d, 200, [3], 1)

filter of size  6 * 597 was created

filter of size  6 * 597 was created



In [30]:
clf_C = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                  max_depth=1, random_state=0).fit(X_train_transformed_C, y_C)
clf_C.score(X_train_transformed_C, y_C)

0.8518518518518519

In [31]:
clf_C.apply(X_train_transformed_C)

array([[[2., 2., 1., ..., 2., 1., 1.],
        [1., 2., 1., ..., 2., 1., 1.],
        [1., 1., 2., ..., 2., 1., 1.],
        ...,
        [2., 2., 1., ..., 2., 1., 1.],
        [2., 2., 1., ..., 2., 1., 1.],
        [2., 2., 1., ..., 2., 1., 1.]],

       [[2., 1., 1., ..., 2., 1., 1.],
        [1., 1., 1., ..., 2., 1., 1.],
        [2., 2., 2., ..., 2., 1., 1.],
        ...,
        [2., 1., 1., ..., 2., 1., 1.],
        [2., 2., 1., ..., 2., 1., 1.],
        [2., 1., 1., ..., 2., 1., 1.]],

       [[2., 2., 1., ..., 2., 1., 1.],
        [1., 2., 1., ..., 2., 1., 1.],
        [1., 1., 2., ..., 2., 1., 1.],
        ...,
        [2., 2., 1., ..., 2., 1., 1.],
        [2., 2., 1., ..., 2., 1., 1.],
        [2., 1., 1., ..., 2., 1., 1.]],

       ...,

       [[2., 2., 1., ..., 2., 1., 1.],
        [1., 1., 1., ..., 2., 1., 1.],
        [1., 1., 2., ..., 2., 1., 1.],
        ...,
        [2., 1., 1., ..., 2., 1., 1.],
        [2., 1., 1., ..., 2., 1., 1.],
        [2., 1., 1., ..., 2., 1.

# Accuracy for train Data:

In [32]:
# Accuracy:
clf_train_pred_C=clf_C.predict(X_train_transformed_C)
clf_acc_score_C = accuracy_score(y_C, clf_train_pred_C)
print("Accuracy_score:",clf_acc_score_C)

train_C_f1 = f1_score(y_C, clf_train_pred_C, average='weighted')
train_C_f2 = f1_score(y_C, clf_train_pred_C, average='macro')
train_C_f3 = f1_score(y_C, clf_train_pred_C, average='micro')
print("f1_score:Weighted:",train_C_f1)
print("f1_score:macro:",train_C_f2)
print("f1_score:micro:",train_C_f3)

Accuracy_score: 0.8518518518518519
f1_score:Weighted: 0.8473957844700878
f1_score:macro: 0.8473957844700878
f1_score:micro: 0.8518518518518519


In [33]:
roc_auc_score(y_C, clf_C.predict_proba(X_train_transformed_C), multi_class='ovr')

0.9180695847362511

# Accuracy for test Data:

In [34]:
# Accuracy:
clf_test_pred_C=clf_C.predict(X_test_transformed_C)
clf_acc_score_test_C = accuracy_score(y_C_test, clf_test_pred_C)
print("Accuracy_score:",clf_acc_score_test_C)

# F1-Score:
test_C_f1 = f1_score(y_C_test, clf_test_pred_C, average='weighted')
test_C_f2 = f1_score(y_C_test, clf_test_pred_C, average='macro')
test_C_f3 = f1_score(y_C_test, clf_test_pred_C, average='micro')
print("f1_score:Weighted:",test_C_f1)
print("f1_score:macro:",test_C_f2)
print("f1_score:micro:",test_C_f3)

Accuracy_score: 0.5138888888888888
f1_score:Weighted: 0.4891534391534391
f1_score:macro: 0.48915343915343906
f1_score:micro: 0.5138888888888888


In [35]:
roc_auc_score(y_C_test, clf_C.predict_proba(X_test_transformed_C), multi_class='ovr')

0.7544191919191919

# For the Epilepsy Data:

In [36]:
X_E,y_E= load_UCR_UEA_dataset('Epilepsy', split="train", return_X_y=True)
X_E_test,y_E_test= load_UCR_UEA_dataset('Epilepsy', split="test", return_X_y=True)
X_E_3d = from_nested_to_3d_numpy(X_E)
X_E_3d.shape
X_E_test_3d= from_nested_to_3d_numpy(X_E_test)
X_E_test_3d.shape

print(X_E_3d.shape)
print(X_E_test_3d.shape)

(137, 3, 206)
(138, 3, 206)


In [37]:
X_train_transformed_E = transform_data(X_E_3d, 16, [3], 1)
X_test_transformed_E = transform_data(X_E_test_3d, 16, [3], 1)

filter of size  3 * 158 was created

filter of size  3 * 158 was created



In [38]:
clf_E = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                  max_depth=1, random_state=0).fit(X_train_transformed_E, y_E)
clf_E.score(X_train_transformed_E, y_E)

1.0

In [39]:
clf_E.apply(X_train_transformed_E)

array([[[2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [1., 2., 2., 2.],
        ...,
        [2., 1., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.]],

       [[1., 1., 1., 1.],
        [1., 2., 1., 1.],
        [1., 1., 1., 1.],
        ...,
        [1., 1., 1., 1.],
        [2., 1., 1., 1.],
        [1., 1., 1., 1.]],

       [[1., 1., 1., 1.],
        [1., 2., 1., 1.],
        [1., 2., 1., 1.],
        ...,
        [1., 1., 1., 1.],
        [2., 1., 1., 2.],
        [2., 1., 1., 2.]],

       ...,

       [[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        ...,
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]],

       [[1., 2., 1., 2.],
        [1., 2., 2., 2.],
        [1., 2., 2., 1.],
        ...,
        [2., 1., 1., 2.],
        [2., 1., 1., 2.],
        [2., 1., 2., 2.]],

       [[1., 2., 1., 2.],
        [2., 2., 2., 2.],
        [1., 2., 2., 1.],
        ...,
        [2., 1., 1., 2.],
        [2., 1.

# Accuracy for Train Data:

In [40]:
# Accuracy:
clf_train_pred_E=clf_E.predict(X_train_transformed_E)
clf_acc_score_E = accuracy_score(y_E, clf_train_pred_E)
print("Accuracy_score:",clf_acc_score_E)

train_E_f1 = f1_score(y_E, clf_train_pred_E, average='weighted')
train_E_f2 = f1_score(y_E, clf_train_pred_E, average='macro')
train_E_f3 = f1_score(y_E, clf_train_pred_E, average='micro')
print("f1_score:Weighted:",train_E_f1)
print("f1_score:macro:",train_E_f2)
print("f1_score:micro:",train_E_f3)

Accuracy_score: 1.0
f1_score:Weighted: 1.0
f1_score:macro: 1.0
f1_score:micro: 1.0


In [41]:
roc_auc_score(y_E, clf_E.predict_proba(X_train_transformed_E), multi_class='ovr')

1.0

# Accuracy for Test Data:


In [42]:
# Accuracy:
clf_test_pred_E=clf_E.predict(X_test_transformed_E)
clf_acc_score_test_E = accuracy_score(y_E_test, clf_test_pred_E)
print("Accuracy_score:",clf_acc_score_test_E)

# F1-Score:
test_E_f1 = f1_score(y_E_test, clf_test_pred_E, average='weighted')
test_E_f2 = f1_score(y_E_test, clf_test_pred_E, average='macro')
test_E_f3 = f1_score(y_E_test, clf_test_pred_E, average='micro')
print("f1_score:Weighted:",test_E_f1)
print("f1_score:macro:",test_E_f2)
print("f1_score:micro:",test_E_f3)

Accuracy_score: 0.2898550724637681
f1_score:Weighted: 0.2827929970559522
f1_score:macro: 0.2847686707030478
f1_score:micro: 0.2898550724637681


In [43]:
roc_auc_score(y_E_test, clf_E.predict_proba(X_test_transformed_E), multi_class='ovr')

0.5117874602898749

# For the FingerMovements Data:


In [44]:
X_F,y_F= load_UCR_UEA_dataset('FingerMovements',return_X_y=True)
X_F_Train,X_F_test,y_F_Train,y_F_test= train_test_split(X_F,y_F, test_size=0.5, random_state=0)
X_F_3d = from_nested_to_3d_numpy(X_F_Train)
X_F_3d.shape
X_F_test_3d= from_nested_to_3d_numpy(X_F_test)
X_F_test_3d.shape

print(X_F_3d.shape)
print(X_F_test_3d.shape)

(208, 28, 50)
(208, 28, 50)


In [45]:
X_train_transformed_F = transform_data(X_F_3d, 16, [3], 1)
X_test_transformed_F = transform_data(X_F_test_3d, 16, [3], 1)

filter of size  28 * 2 was created

filter of size  28 * 2 was created



In [46]:
clf_F = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                  max_depth=1, random_state=0).fit(X_train_transformed_F, y_F_Train)
clf_F.score(X_train_transformed_F, y_F_Train)

0.9711538461538461

In [47]:
clf_F.apply(X_train_transformed_F)

array([[[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]],

       [[2.],
        [2.],
        [1.],
        ...,
        [2.],
        [2.],
        [2.]],

       [[2.],
        [2.],
        [1.],
        ...,
        [2.],
        [2.],
        [2.]],

       ...,

       [[2.],
        [2.],
        [1.],
        ...,
        [2.],
        [2.],
        [2.]],

       [[2.],
        [2.],
        [1.],
        ...,
        [1.],
        [2.],
        [2.]],

       [[2.],
        [2.],
        [1.],
        ...,
        [1.],
        [2.],
        [2.]]])

# Accuracy for Train Data:


In [48]:
# Accuracy:
clf_train_pred_F=clf_F.predict(X_train_transformed_F)
clf_acc_score_F = accuracy_score(y_F_Train, clf_train_pred_F)
print("Accuracy_score:",clf_acc_score_F)

train_F_f1 = f1_score(y_F_Train, clf_train_pred_F, average='weighted')
train_F_f2 = f1_score(y_F_Train, clf_train_pred_F, average='macro')
train_F_f3 = f1_score(y_F_Train, clf_train_pred_F, average='micro')
print("f1_score:Weighted:",train_F_f1)
print("f1_score:macro:",train_F_f2)
print("f1_score:micro:",train_F_f3)

Accuracy_score: 0.9711538461538461
f1_score:Weighted: 0.9711485102023393
f1_score:macro: 0.9711431742508324
f1_score:micro: 0.9711538461538461


# Accuracy for Test Data:


In [49]:
# Accuracy:
clf_test_pred_F=clf_F.predict(X_test_transformed_F)
clf_acc_score_test_F = accuracy_score(y_F_test, clf_test_pred_F)
print("Accuracy_score:",clf_acc_score_test_F)

# F1-Score:
test_F_f1 = f1_score(y_F_test, clf_test_pred_F, average='weighted')
test_F_f2 = f1_score(y_F_test, clf_test_pred_F, average='macro')
test_F_f3 = f1_score(y_F_test, clf_test_pred_F, average='micro')
print("f1_score:Weighted:",test_F_f1)
print("f1_score:macro:",test_F_f2)
print("f1_score:micro:",test_F_f3)

Accuracy_score: 0.44711538461538464
f1_score:Weighted: 0.4471281642334274
f1_score:macro: 0.44710260499734183
f1_score:micro: 0.44711538461538464


In [50]:
roc_auc_score(y_F_test, clf_F.predict_proba(X_test_transformed_F)[:,1])

0.43444290337494224

# For the Handwriting Data Set:

In [51]:
X_H,y_H= load_UCR_UEA_dataset('Handwriting', return_X_y=True)
X_H_Train,X_H_test,y_H_Train,y_H_test= train_test_split(X_H,y_H, test_size=0.5, random_state=0)
X_H_3d = from_nested_to_3d_numpy(X_H_Train)
X_H_3d.shape
X_H_test_3d= from_nested_to_3d_numpy(X_H_test)
X_H_test_3d.shape

print(X_H_3d.shape)
print(X_H_test_3d.shape)

(500, 3, 152)
(500, 3, 152)


In [52]:
X_train_transformed_H = transform_data(X_H_3d, 10, [3], 1)
X_test_transformed_H = transform_data(X_H_test_3d, 10, [3], 1)

filter of size  3 * 122 was created

filter of size  3 * 122 was created



In [53]:
clf_H = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                  max_depth=1, random_state=0).fit(X_train_transformed_H, y_H_Train)
clf_H.score(X_train_transformed_H, y_H_Train)

0.036

In [54]:
clf_H.apply(X_train_transformed_H)

array([[[1., 1., 1., ..., 1., 2., 1.],
        [2., 1., 2., ..., 1., 2., 1.],
        [2., 1., 1., ..., 1., 2., 1.],
        ...,
        [1., 1., 1., ..., 1., 2., 1.],
        [1., 1., 1., ..., 1., 2., 1.],
        [1., 1., 1., ..., 1., 2., 1.]],

       [[1., 1., 1., ..., 1., 2., 1.],
        [2., 1., 2., ..., 1., 2., 1.],
        [2., 1., 1., ..., 1., 2., 1.],
        ...,
        [1., 1., 1., ..., 1., 2., 1.],
        [1., 1., 1., ..., 1., 2., 1.],
        [1., 1., 1., ..., 1., 2., 1.]],

       [[1., 1., 2., ..., 2., 2., 1.],
        [2., 1., 2., ..., 1., 2., 1.],
        [2., 1., 2., ..., 1., 2., 1.],
        ...,
        [1., 1., 2., ..., 2., 2., 2.],
        [1., 1., 2., ..., 2., 2., 2.],
        [1., 1., 2., ..., 2., 2., 2.]],

       ...,

       [[1., 1., 1., ..., 1., 2., 1.],
        [2., 1., 2., ..., 1., 2., 1.],
        [2., 1., 1., ..., 1., 2., 1.],
        ...,
        [1., 1., 1., ..., 1., 2., 1.],
        [1., 1., 1., ..., 1., 2., 1.],
        [1., 1., 1., ..., 1., 2.

# Accuracy for Train Data:

In [55]:
# Accuracy:
clf_train_pred_H=clf_H.predict(X_train_transformed_H)
clf_acc_score_H = accuracy_score(y_H_Train, clf_train_pred_H)
print("Accuracy_score:",clf_acc_score_H)

train_H_f1 = f1_score(y_H_Train, clf_train_pred_H, average='weighted')
train_H_f2 = f1_score(y_H_Train, clf_train_pred_H, average='macro')
train_H_f3 = f1_score(y_H_Train, clf_train_pred_H, average='micro')
print("f1_score:Weighted:",train_H_f1)
print("f1_score:macro:",train_H_f2)
print("f1_score:micro:",train_H_f3)

Accuracy_score: 0.036
f1_score:Weighted: 0.01800859242905443
f1_score:macro: 0.017411040357140122
f1_score:micro: 0.036


In [56]:
roc_auc_score(y_H_Train, clf_H.predict_proba(X_train_transformed_H), multi_class='ovr')

0.49737676775918715

# Accuracy for Test Data:

In [57]:
# Accuracy:
clf_test_pred_H=clf_H.predict(X_test_transformed_H)
clf_acc_score_test_H = accuracy_score(y_H_test, clf_test_pred_H)
print("Accuracy_score:",clf_acc_score_test_H)

# F1-Score:
test_H_f1 = f1_score(y_H_test, clf_test_pred_H, average='weighted')
test_H_f2 = f1_score(y_H_test, clf_test_pred_H, average='macro')
test_H_f3 = f1_score(y_H_test, clf_test_pred_H, average='micro')
print("f1_score:Weighted:",test_H_f1)
print("f1_score:macro:",test_H_f2)
print("f1_score:micro:",test_H_f3)

Accuracy_score: 0.038
f1_score:Weighted: 0.005689746222678598
f1_score:macro: 0.0055136480066801325
f1_score:micro: 0.038


In [58]:
roc_auc_score(y_H_test, clf_H.predict_proba(X_test_transformed_H), multi_class='ovr')

0.4978271207564659

# For Basic Motions:


In [59]:
X_B, X_B_test, y_B, y_B_test = load_basic_motions(return_X_y=True)


print(X_B.shape)
print(X_B_test.shape)

(40, 6, 100)
(40, 6, 100)


In [60]:
X_train_transformed_B = transform_data(X_B, 16, [3], 1)
X_test_transformed_B = transform_data(X_B_test, 16, [3], 1)

filter of size  6 * 52 was created

filter of size  6 * 52 was created



In [61]:
clf_B = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                  max_depth=1, random_state=0).fit(X_train_transformed_B, y_B)
clf_B.score(X_train_transformed_B, y_B)

1.0

# Accuracy for Train Data:

In [62]:
# Accuracy:
clf_train_pred_B=clf_B.predict(X_train_transformed_B)
clf_acc_score_B = accuracy_score(y_B, clf_train_pred_B)
print("Accuracy_score:",clf_acc_score_B)

train_B_f1 = f1_score(y_B, clf_train_pred_B, average='weighted')
train_B_f2 = f1_score(y_B, clf_train_pred_B, average='macro')
train_B_f3 = f1_score(y_B, clf_train_pred_B, average='micro')
print("f1_score:Weighted:",train_B_f1)
print("f1_score:macro:",train_B_f2)
print("f1_score:micro:",train_B_f3)

Accuracy_score: 1.0
f1_score:Weighted: 1.0
f1_score:macro: 1.0
f1_score:micro: 1.0


In [63]:
roc_auc_score(y_B, clf_B.predict_proba(X_train_transformed_B), multi_class='ovr')

1.0

# Accuracy for Test Data:

In [64]:
# Accuracy:
clf_test_pred_B=clf_B.predict(X_test_transformed_B)
clf_acc_score_test_B = accuracy_score(y_B_test, clf_test_pred_B)
print("Accuracy_score:",clf_acc_score_test_B)

# F1-Score:
test_B_f1 = f1_score(y_B_test, clf_test_pred_B, average='weighted')
test_B_f2 = f1_score(y_B_test, clf_test_pred_B, average='macro')
test_B_f3 = f1_score(y_B_test, clf_test_pred_B, average='micro')
print("f1_score:Weighted:",test_B_f1)
print("f1_score:macro:",test_B_f2)
print("f1_score:micro:",test_B_f3)

Accuracy_score: 0.325
f1_score:Weighted: 0.3096590909090909
f1_score:macro: 0.3096590909090909
f1_score:micro: 0.325


In [65]:
roc_auc_score(y_B_test, clf_B.predict_proba(X_test_transformed_B), multi_class='ovr')

0.6475000000000001